In [2]:
from learn import *
import pandas as pd

In [4]:
dataframes = []
for i in range(2002, 2022):
    df_temp = tableMethod(year=i, input_id='team_stats', url2='https://www.pro-football-reference.com/years/{}/')
    df_temp['Year'] = i
    df_temp2 = df_temp[0:32]
    dataframes.append(df_temp2)
df_off = pd.concat(dataframes)
df_off

,Tm,G,PF,Yds,Ply,Y/P,TO,FL,1stD,Cmp,...,TD,Y/A,1stD,Pen,Yds,1stPy,Sc%,TO%,EXP,Year
0,Kansas City Chiefs,16,467,6000,965,6.2,15,2,343,292,...,26,5.1,140,75,700,24,42.7,7.9,145.45,2002
1,Oakland Raiders,16,450,6237,1069,5.8,19,9,366,418,...,21,4.3,113,129,1094,27,42.7,9.4,123.35,2002
2,New Orleans Saints,16,432,5205,1001,5.2,30,15,291,291,...,16,4.1,91,119,911,31,38.5,14.6,-77.44,2002
3,Philadelphia Eagles,16,415,5604,1073,5.2,24,13,311,318,...,15,4.5,124,111,897,23,36.0,12.0,-15.34,2002
4,Atlanta Falcons,16,402,5535,1038,5.3,27,15,316,268,...,23,4.5,123,105,931,30,38.6,13.2,38.40,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,New York Jets,17,310,5208,1036,5.0,27,7,310,357,...,14,4.4,87,103,860,28,29.4,14.4,-32.71,2021
28,Carolina Panthers,17,304,5081,1106,4.6,29,8,322,348,...,17,4.0,118,107,864,30,29.2,14.9,-92.10,2021
29,Houston Texans,17,280,4727,1010,4.7,22,7,266,354,...,8,3.4,77,114,978,27,26.6,11.2,-74.29,2021
30,New York Giants,17,258,4884,1050,4.7,30,10,299,351,...,8,4.0,90,88,674,38,27.5,15.9,-100.13,2021


In [5]:
def_dataframes = []
for i in range(2002, 2022):
    df_temp = tableMethod(year=i, input_id='team_stats', url2='https://www.pro-football-reference.com/years/{}/opp.htm')
    df_temp['Year'] = i
    df_temp2 = df_temp[0:32]
    def_dataframes.append(df_temp2)
df_def = pd.concat(def_dataframes)
df_def

,Tm,G,PA,Yds,Ply,Y/P,TO,FL,1stD,Cmp,...,TD,Y/A,1stD,Pen,Yds,1stPy,Sc%,TO%,EXP,Year
0,Tampa Bay Buccaneers,16,196,4044,963,4.2,38,7,236,259,...,8,3.8,79,93,749,26,17.8,18.3,284.70,2002
1,Philadelphia Eagles,16,241,4754,1009,4.7,38,23,272,333,...,5,4.3,83,112,969,29,20.6,18.0,173.69,2002
2,New York Giants,16,279,4949,985,5.0,25,14,266,311,...,12,4.5,87,120,897,17,27.0,12.4,63.16,2002
3,Miami Dolphins,16,301,4656,974,4.8,30,9,285,294,...,9,3.8,86,106,826,37,30.3,14.4,117.76,2002
4,Carolina Panthers,16,302,4646,1022,4.5,33,16,271,293,...,12,3.7,95,98,782,21,25.5,14.2,123.24,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,Jacksonville Jaguars,17,457,6002,1068,5.6,9,2,351,377,...,22,4.3,124,104,879,29,45.3,5.2,-154.57,2021
28,Atlanta Falcons,17,459,6194,1114,5.6,20,8,383,391,...,19,4.3,143,118,956,26,44.7,11.2,-136.74,2021
29,Los Angeles Chargers,17,459,6122,1103,5.6,21,10,383,357,...,22,4.6,144,99,835,40,45.3,11.2,-122.86,2021
30,Detroit Lions,17,467,6456,1092,5.9,19,8,372,359,...,19,4.4,135,87,818,26,47.8,10.6,-176.03,2021


In [8]:
df_super_bowl_winners = pd.read_csv('super-bowl-winners.csv')
df_super_bowl_winners

,Date,SB,Winner,Pts,Loser,Pts.1,MVP,Stadium,City,State
0,Feb 13 2022,LVI (56),Los Angeles Rams,23,Cincinnati Bengals,20,Cooper Kupp,SoFi Stadium,Inglewood,California
1,Feb 7 2021,LV (55),Tampa Bay Buccaneers,31,Kansas City Chiefs,9,Tom Brady,Raymond James Stadium,Tampa,Florida
2,Feb 2 2020,LIV (54),Kansas City Chiefs,31,San Francisco 49ers,20,Patrick Mahomes,Hard Rock Stadium,Miami Gardens,Florida
3,Feb 3 2019,LIII (53),New England Patriots,13,Los Angeles Rams,3,Julian Edelman,Mercedes-Benz Stadium,Atlanta,Georgia
4,Feb 4 2018,LII (52),Philadelphia Eagles,41,New England Patriots,33,Nick Foles,U.S. Bank Stadium,Minneapolis,Minnesota
5,Feb 5 2017,LI (51),New England Patriots,34,Atlanta Falcons,28,Tom Brady,NRG Stadium,Houston,Texas
6,Feb 7 2016,50,Denver Broncos,24,Carolina Panthers,10,Von Miller,Levi's Stadium,Santa Clara,California
7,Feb 1 2015,XLIX (49),New England Patriots,28,Seattle Seahawks,24,Tom Brady,University of Phoenix Stadium,Glendale,Arizona
8,Feb 2 2014,XLVIII (48),Seattle Seahawks,43,Denver Broncos,8,Malcolm Smith,MetLife Stadium,East Rutherford,New Jersey
9,Feb 3 2013,XLVII (47),Baltimore Ravens,34,San Francisco 49ers,31,Joe Flacco,Mercedes-Benz Superdome,New Orleans,Louisiana
